# Pytorch Tutorial

by Eunjikeam

[2019-12-18] 신경망 정의하기 tutorial

-----------------------

**reference** <br>
- [pytorch tutorial](https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/blitz/neural_networks_tutorial.html)<br>
- [book]자연어처리 딥러딩 캠프(파이토치 편)

## 1. 신경망 (Neural Networks)
신경망은 `torch.nn`패키지를 사용하여 생성할 수 있다. 파이토치는 `nn.Module`이라는 클래스를 제공하여 사용자가 그 위에 필요한 모델 구조를 구현할 수 있다록 한다. `nn`은 모델을 정의하고 미분하는데 `autograd`를 사용한다. `nn.Module`은 계층(layer)과 output을 반환하는 `foward(input)`메서드를 포함하고 있다. 

신경망의 전형적인 학습 과정은 다음과 같다. 

- 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의한다. 
- 데이터 셋(dataset) 입력을 반복한다. 
- 입력을 신경망에서 처리한다. 
- 손실(loss)을 계산한다. 
- gradient를 신경망의 매개변수들에 역으로 전파한다. 
- 신경망의 가중치를 갱신한다(weight = weight - learning_rate * gradient)

### nn.Module
`nn.Module`을 상속한 사용자 정의 클래스는 내부에 `nn.Module`을 상속한 클래스 객체를 선언하여 소유할 수 있다. **즉, `nn.Module` 상속 객체 안에 `nn.Module` 상속 객체를 선언하여 변수로 사용할 수 있다.** 그리고 nn.Module의 forward()함수를 오버라이드 하여 피드포워드를 구현할 수 있다. 이 외에도 nn.Module의 특징을 이용하여 한번에 신경망 가중치 파라미터들을 저장 및 불러오기를 수행할 수 있다. 

아래는 숫자 이미지를 분류하는 신경망을 예제로 만들어 본다. 
![convnet](https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/_images/mnist.png)

In [16]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)   # convolution layer
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  #linear layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수만 정의하게 되면 `backward`함수는 `autograd`함수를 사용하여 자동으로 정의한다. `forward`함수에서는 어떠한 tensor연산을 사용해도 된다. 

모델의 학습 간으한 매개변수들은 `net.parameters()`에 의해 반환된다.

In [17]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


forward의 입력은 `autograd.Variable` 이고, 출력 또한 마찬가지입니다. 

Note: 이 신경망(LeNet)의 입력은 32x32입니다. 이 신경망에 MNIST 데이터셋을 사용하기 위해서는, 데이터셋의 이미지를 32x32로 크기를 변경해야 합니다.

In [18]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

tensor([[-0.1122, -0.0254,  0.0934, -0.0213,  0.1333, -0.0958, -0.1290, -0.0406,
          0.0492,  0.1013]], grad_fn=<AddmmBackward>)


In [19]:
net.zero_grad()
out.backward(torch.rand(1,10))

Note

torch.nn 은 미니 배치(mini-batch)만 지원합니다. torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니배치만을 입력으로 받습니다.

예를 들어, nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, input.unsqueeze(0) 을 사용해서 가짜 차원을 추가합니다.

-------------------

### 객체 정리

- **`torch.Tensor`** : 다차원 배열
- **`autograd.Variable`** : Tensor를 포함한 모든 연산을 기록한다. Tensor와 동일한 API를 갖고 있으며, backward()와 같이 추가된 것들도 있다. 또한 tensor에 대한 gradient 또한 가지고 있다. 
- **`nn.Module`** : 신경망 모듈. 매개변수를 캡슐화 하는 간편한 방법으로 몇가지 작업을 위한 헬퍼를 제공한다. 
- **`nn.parameter`** : 변수의 한 종류로, `Module`에 속성으로 할당될 때 **자동으로 매개변수로 등록된다. **
- **`autograd.Function`** : `autograd` 연산의 전방향과 역방향 정의 를 구현. 모든 Variable 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Variable 을 생성하고 이력(History)을 부호화 하는 함수들과 연결하고 있다.

--------------------

## 손실함수(Loss Funtion)

손실함수는 (output, target)을 한쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 떨어져 있는지를 추정하는 값을 계산한다. 

nn패키지에는 여러가지의 손실함수들이 존대한다. 간단한 손실함수로는 출력과 대상간의 평균오차를 계산하는 MSE함수 `nn.MSEloss`가 있다. 

In [27]:
output = net(input)
target = Variable(torch.arange(1., 11.))  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.4376, grad_fn=<MseLossBackward>)


grad_fn을 통해 loss를 역방향으로 따라가며 추적할 수 있다. 

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  <br>
      -> view -> linear -> relu -> linear -> relu -> linear<br>
      -> MSELoss<br>
      -> loss<br>

In [29]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 역전파(Backprop)

오차(error)를 역전파 하기 위해 할 일은 loss.backward()이 전부다. **기존 gradient을 지우는 작업이 필요하다.** 그렇지 않으면 변화도가 기존의 것에 누적됨.

In [30]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0215, -0.0045,  0.0247,  0.0901, -0.0493,  0.0504])


## 가중치 갱신하기

실제로 가장 많이 사용되는 단순한 갱신 규칙은 확률적 경사 하강법(SGD; Stochastic Gradient Descent)이다. 

**weight = weight - learning_rate + gradient**

In [31]:
## 가중치 갱신
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [32]:
## 다른 가중치 갱신 방법
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)에서는 다음과 같습니다:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
